#### Set the enviroments.

In [1]:
(ext:chdir "~/Development/clasp/extensions/cando/src/lisp/tests/alanine-demo/" t)

-1

In [2]:
(start-swank)

swank-loader -> /Users/tuj24515/Development/slime/swank-loader.lisp

"Read: (DEFPACKAGE :SWANK-LOADER (:USE :CL) (:EXPORT :INIT :DUMP-IMAGE :LIST-FASLS :*SOURCE-DIRECTORY* :*FASL-DIRECTORY*))
" 
"Read: (IN-PACKAGE :SWANK-LOADER)
" 
"Read: (DEFVAR *SOURCE-DIRECTORY* (MAKE-PATHNAME :NAME NIL :TYPE NIL :DEFAULTS (OR *LOAD-PATHNAME* *DEFAULT-PATHNAME-DEFAULTS*)) The directory where to look for the source.)
" 
"Read: (DEFPARAMETER *SYSDEP-FILES* (QUOTE ((SWANK CLASP) (SWANK GRAY))))
" 
"Read: (DEFPARAMETER *IMPLEMENTATION-FEATURES* (QUOTE (:ALLEGRO :LISPWORKS :SBCL :CLOZURE :CMU :CLISP :CCL :CORMAN :CORMANLISP :ARMEDBEAR :GCL :ECL :SCL :MKCL :CLASP)))
" 
"Read: (DEFPARAMETER *OS-FEATURES* (QUOTE (:MACOSX :LINUX :WINDOWS :MSWINDOWS :WIN32 :SOLARIS :DARWIN :SUNOS :HPUX :UNIX)))
" 
"Read: (DEFPARAMETER *ARCHITECTURE-FEATURES* (QUOTE (:POWERPC :PPC :X86 :X86-64 :X86_64 :AMD64 :I686 :I586 :I486 :PC386 :IAPX386 :SPARC64 :SPARC :HPPA64 :HPPA :ARM :ARMV5L :ARMV6L :ARMV7L :ARM64 :PENTIUM3 :PENTIUM4 

T

In [3]:
(add-path "~/Development/amber16/dat/leap/cmd/")
(add-path "~/Development/amber16/dat/leap/lib/")
(add-path "~/Development/amber16/dat/leap/parm/")
(add-path "~/Development/amber16/dat/leap/prep/")
(add-path "~/Development/amber16/dat/antechamber/")

(#P"/Users/tuj24515/Development/amber16/dat/antechamber/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/prep/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/parm/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/lib/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/cmd/")

#### Load the amber force field.

In [4]:
(load-atom-type-rules "ATOMTYPE_AMBER.DEF")

(#<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [5]:
(source "leaprc.protein.ff14SB")

SIMPLE-WARNING: 
add-atom-types doesn't do anything
SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


T

In [6]:
(:= *ff* (leap.core::merged-force-field))

SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb
SIMPLE-WARNING: 
Implement merge of FFVdwDb


*FF*

#### Load the alanine from Chemdraw file.

In [7]:
(defparameter *chemagg* (cando:load-chem-draw-aggregate "alanine2.cdxml"))

*CHEMAGG*

In [8]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

#### Gather atoms to the center.

In [9]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *chemagg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *chemagg*)))

C      S


NIL

#### Set the bounding box.

In [10]:
(chem:set-property *chemagg* :bounding-box '(50.0 50.0 50.0))

NIL

#### Assign the atom types of the force field to the aggregate.

In [11]:
(chem:assign-types (chem:get-types *ff*) *chemagg*)

NIL

####  Assign force-field parameters and minimize the energy of the molecule.

In [12]:
(cando:jostle *chemagg*)

NIL

In [13]:
(energy:minimize *chemagg* :force-field *ff*)

======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000         211581.688
 minSDnP       0     2  0.000338   0.0       89833285.047          58077.277
 minSDnP       0     3  0.000397   0.0       45562264.568          34394.140
 minSDnP       0     4  0.000390   0.0       30312261.576          24904.205
 minSDnP       0     5  0.000438   0.0       21392333.759          21015.344
 minSDnP       0     6  0.000400   0.0       15567436.226          16370.499
 minSDnP       0     7  0.000449   0.0       11603328.605          14430.025
 minSDnP       0     8  0.000408   0.0        8815353.061          11626.358
 minSDnP       0     9  0.000457   0.0        6773363.510          10507.854
 minSDnP       0    10  0.000412   0.0        5269481.957           8674.378
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0    11  0.000456

#<ENERGY-FUNCTION >

#### Visualize the minimized structure and demonstrate that the force field parameters lead to the construction of a chemically

#### reasonable structure.

In [14]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

#### Assign atom types and change one of them for alanine atom type definition.

In [15]:
(chem:assign-types (chem:get-types *ff*) *chemagg*)

NIL

In [16]:
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

#### Create the topology file and coordinate file.

In [17]:
(leap.topology:save-amber-parm-format *chemagg* "chemagg.top" "chemagg.crd" *ff* :assign-types nil)

atom1: #<ATOM :O/:O> atom2: #<ATOM :C/:C> i1: 0 i2: 3
 atom1: #<ATOM :C/:C> atom2: #<ATOM :NH/:N> i1: 6 i2: 9
 atom1: #<ATOM :C/:C> atom2: #<ATOM :CH3/:C> i1: 6 i2: 15
 atom1: #<ATOM :C/:C> atom2: #<ATOM :C/:C> i1: 6 i2: 3
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 9 i2: 51
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :HH/:H> i1: 9 i2: 27
 atom1: #<ATOM :H/:H> atom2: #<ATOM :C/:C> i1: 12 i2: 6
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 18 i2: 15
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 21 i2: 15
 atom1: #<ATOM :HH33/:H> atom2: #<ATOM :CH3/:C> i1: 24 i2: 15
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :NH/:N> i1: 30 i2: 33
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 33 i2: 3
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 36 i2: 30
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 39 i2: 30
 atom1: #<ATOM :HH33/:H> atom2: #<ATOM :CH3/:C> i1: 42 i2: 30
 atom1: #<ATOM :HH/:H> atom2: #<ATOM :NH/:N> i1: 45 i2: 33
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C/:C> i1: 48 i2: 51
 

0

#### Load the water box and make solvent box.

In [65]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

In [66]:
(print ~tip3pfbox)


AGGREGATE 

AGGREGATE

In [67]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [68]:
(nglv::show-aggregate *solvent*)

A Jupyter Widget

In [69]:
(chem:assign-types *ff* *solvent*)

NIL

In [70]:
(:= *newagg* (leap.solvate::tool-solvate-and-shell *chemagg* *solvent* '(10.0 10.0 10.0)))

Total bounding box for atom centers:  28.633287466224004d0 24.591801914236044d0 23.95789660974532d0


*NEWAGG*

In [71]:
(nglv::show-aggregate *newagg*)

A Jupyter Widget

In [76]:
(chem:assign-types (chem:get-types *ff*) *newagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *newagg*)

NIL

In [77]:
(chem:map-atoms 'list (lambda (a) (chem:get-type a)) *newagg*)

(:CT :N :H1 :H1 :H1 :H :O :C :CT :HC :HC :HC :C :CX :O :N :H1 :CT :HC :HC :HC
 :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW
 :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H
 :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H
 :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW
 :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H
 :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H
 :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW
 :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H
 :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H
 :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW
 :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H
 :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H :H :OW :H 

In [79]:
(leap.topology:save-amber-parm-format *newagg* "chemagg_water.top" "chemagg_water.crd" *ff* :assign-types nil)

SIMPLE-WARNING: 
Could not find stretch parameter between #<ATOM :H1/:H> and #<ATOM :O/:O>
SIMPLE-WARNING: 
Could not find stretch parameter between #<ATOM :H2/:H> and #<ATOM :O/:O>
SIMPLE-WARNING: 
Could not find stretch parameter between #<ATOM :H2/:H> and #<ATOM :H1/:H>
SIMPLE-WARNING: 
Could not find stretch parameter between #<ATOM :H1/:H> and #<ATOM :O/:O>
SIMPLE-WARNING: 
Could not find stretch parameter between #<ATOM :H2/:H> and #<ATOM :O/:O>
SIMPLE-WARNING: 
Could not find stretch parameter between #<ATOM :H2/:H> and #<ATOM :H1/:H>
SIMPLE-WARNING: 
Could not find stretch parameter between #<ATOM :H1/:H> and #<ATOM :O/:O>
SIMPLE-WARNING: 
Could not find stretch parameter between #<ATOM :H2/:H> and #<ATOM :O/:O>
SIMPLE-WARNING: 
Could not find stretch parameter between #<ATOM :H2/:H> and #<ATOM :H1/:H>
SIMPLE-WARNING: 
Could not find stretch parameter between #<ATOM :H1/:H> and #<ATOM :O/:O>
SIMPLE-WARNING: 
Could not find stretch parameter between #<ATOM :H2/:H> and #<ATOM :O/

NIL

In [26]:
(:= *e* (chem:make-energy-function *chemagg* *ff*))

*E*

In [27]:
(energy::minimize-energy-function *e*)

======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000         214882.471
 minSDnP       0     2  0.000338   0.0       91066421.566          57164.149
 minSDnP       0     3  0.000411   0.0       46729393.274          34918.438
 minSDnP       0     4  0.000390   0.0       31017900.052          25303.344
 minSDnP       0     5  0.000441   0.0       21746608.073          21421.803
 minSDnP       0     6  0.000398   0.0       15723243.933          16518.380
 minSDnP       0     7  0.000451   0.0       11667405.708          14585.389
 minSDnP       0     8  0.000407   0.0        8834537.726          11651.094
 minSDnP       0     9  0.000461   0.0        6761639.350          10560.893
 minSDnP       0    10  0.000412   0.0        5248635.296           8630.448
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0    11  0.000463

NIL

In [35]:
(save-mol2 *chemagg* "alanine-min.mol2")

Saving matter to /Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/alanine-min.mol2


NIL

In [28]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

In [8]:
(nglv::show-aggregate ~tip3pfbox)

A Jupyter Widget

In [14]:
(apropos "loadpdb")

LEAP.PDB:LOADPDB  Function


NIL

In [7]:
(:= *r* (chem:build-residue ~ala))

*R*

In [9]:
(chem:get-type (chem:atom-with-name *r* :CA))

:CX

In [63]:
(chem:map-atoms 'list (lambda (a) (chem:get-type a)) *chemagg*)

(:CT :N :H1 :H1 :H1 :H :O :C :CT :HC :HC :HC :C :CX :O :N :H1 :CT :HC :HC :HC
 :H)

In [32]:
(chem:map-atoms 'list (lambda (a) (chem:get-name a)) *chemagg*)

(:CH3 :NH :HH31 :HH32 :HH33 :HH :O :C :CH3 :HH31 :HH32 :HH33 :C :C :O :NH :H
 :CH3 :HH31 :HH32 :HH33 :HH)

In [31]:
(defparameter *met* (chem:make-molecule))
(defparameter *ace* (chem:make-residue :ACE))
(defparameter *ala* (chem:make-residue :ALA))
(defparameter *nme* (chem:make-residue :NME))

(chem:add-matter *met* *ace*)
(chem:add-matter *met* *ala*)
(chem:add-matter *met* *nme*)

(defparameter *h1* (chem:make-atom :HH31 :H))
(defparameter *c1* (chem:make-atom :CH3 :C))
(defparameter *h2* (chem:make-atom :HH32 :H))
(defparameter *h3* (chem:make-atom :HH33 :H))
(defparameter *c2* (chem:make-atom :C :C))
(defparameter *o1* (chem:make-atom :O :O))
(defparameter *n1* (chem:make-atom :N :N))
(defparameter *h4* (chem:make-atom :H :H))
(defparameter *c3* (chem:make-atom :CA :C))
(defparameter *h5* (chem:make-atom :HA :H))
(defparameter *c4* (chem:make-atom :CB :C))
(defparameter *h6* (chem:make-atom :HB1 :H))
(defparameter *h7* (chem:make-atom :HB2 :H))
(defparameter *h8* (chem:make-atom :HB3 :H))
(defparameter *c5* (chem:make-atom :C :C))
(defparameter *o2* (chem:make-atom :O :O))
(defparameter *n2* (chem:make-atom :N :N))
(defparameter *h9* (chem:make-atom :H :H))
(defparameter *c6* (chem:make-atom :CH3 :C))
(defparameter *h10* (chem:make-atom :HH31 :H))
(defparameter *h11* (chem:make-atom :HH32 :H))
(defparameter *h12* (chem:make-atom :HH33 :H))

(chem:add-matter *ace* *h1*)
(chem:add-matter *ace* *c1*)
(chem:add-matter *ace* *h2*)
(chem:add-matter *ace* *h3*)
(chem:add-matter *ace* *c2*)
(chem:add-matter *ace* *o1*)
(chem:add-matter *ala* *n1*)
(chem:add-matter *ala* *h4*)
(chem:add-matter *ala* *c3*)
(chem:add-matter *ala* *h5*)
(chem:add-matter *ala* *c4*)
(chem:add-matter *ala* *h6*)
(chem:add-matter *ala* *h7*)
(chem:add-matter *ala* *h8*)
(chem:add-matter *ala* *c5*)
(chem:add-matter *ala* *o2*)
(chem:add-matter *nme* *n2*)
(chem:add-matter *nme* *h9*)
(chem:add-matter *nme* *c6*)
(chem:add-matter *nme* *h10*)
(chem:add-matter *nme* *h11*)
(chem:add-matter *nme* *h12*)

(chem:bond-to *h1* *c1* :single-bond)
(chem:bond-to *h2* *c1* :single-bond)
(chem:bond-to *h3* *c1* :single-bond)
(chem:bond-to *c1* *c2* :single-bond)
(chem:bond-to *c2* *o1* :double-bond)
(chem:bond-to *c2* *n1* :single-bond)
(chem:bond-to *n1* *h4* :single-bond)
(chem:bond-to *n1* *c3* :single-bond)
(chem:bond-to *c3* *h5* :single-bond)
(chem:bond-to *c3* *c4* :single-bond)
(chem:bond-to *c4* *h6* :single-bond)
(chem:bond-to *c4* *h7* :single-bond)
(chem:bond-to *c4* *h8* :single-bond)
(chem:bond-to *c3* *c5* :single-bond)
(chem:bond-to *c5* *o2* :double-bond)
(chem:bond-to *c5* *n2* :single-bond)
(chem:bond-to *n2* *h9* :single-bond)
(chem:bond-to *n2* *c6* :single-bond)
(chem:bond-to *c6* *h10* :single-bond)
(chem:bond-to *c6* *h11* :single-bond)
(chem:bond-to *c6* *h12* :single-bond)
(defparameter *agg* (chem:make-aggregate))
(chem:add-matter *agg* *met*)
(chem:set-property *agg* :solvent-box '(20.0 30.0 40.0))



NIL

In [30]:
(apropos "center" :chem)

CHEM::CENTER
CHEM::CENTER-NAME
CHEM:GEOMETRIC-CENTER  Function
CHEM:GET-CONFIGURATION-FOR-CENTER  Function
CHEM::NUMBER-OF-CENTERS


NIL

In [32]:
(chem:geometric-center *chemagg*)

[ 106.335 161.046 -0.843269 ]

In [32]:
(energy::setup-amber)


SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


#<FORCE-FIELD >

In [13]:
(cando:jostle *met* 20)
(defparameter *me* (energy:minimize *agg*))

energy-nonbond -> #<ENERGY-NONBOND >
ffnonbond-db -> #<FFNONBOND-DB @0x11ce18a58>
type1 24 type2 24
type1 24 type2 3
type1 24 type2 0
type1 24 type2 48
type1 24 type2 35
type1 24 type2 25
type1 24 type2 18
type1 3 type2 3
type1 3 type2 0
type1 3 type2 48
type1 3 type2 35
type1 3 type2 25
type1 3 type2 18
type1 0 type2 0
type1 0 type2 48
type1 0 type2 35
type1 0 type2 25
type1 0 type2 18
type1 48 type2 48
type1 48 type2 35
type1 48 type2 25
type1 48 type2 18
type1 35 type2 35
type1 35 type2 25
type1 35 type2 18
type1 25 type2 25
type1 25 type2 18
type1 18 type2 18
======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-----Alpha---Dir-------------Energy-----------RMSforce
 minSDnP       0     1  0.000000   0.0              0.000           8455.200
DONE absolute force test:
forceRmsMag(2973.669106).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     2  0.000000   0.0              0.000           2973.669
 minCGnP       0     3  0.0

*ME*

In [53]:
(nglv::show-aggregate *agg*)

A Jupyter Widget

In [65]:
(defparameter *met* (chem:make-molecule))
(defparameter *res* (chem:make-residue :foo))
(defparameter *res1* (chem:make-residue :foo1))

(chem:add-matter *met* *res*)
(chem:add-matter *met* *res1*)

(defparameter *c1* (chem:make-atom :C1 :C))
(defparameter *c2* (chem:make-atom :C2 :C))
(defparameter *c3* (chem:make-atom :C3 :C))
(defparameter *c4* (chem:make-atom :C4 :C))
(defparameter *c5* (chem:make-atom :C5 :C))
(defparameter *c6* (chem:make-atom :C6 :C))
(defparameter *o1* (chem:make-atom :O1 :O))
(defparameter *o2* (chem:make-atom :O2 :O))
(defparameter *o3* (chem:make-atom :O3 :O))
(defparameter *n1* (chem:make-atom :N1 :N))
(defparameter *n2* (chem:make-atom :N2 :N))
(defparameter *h1* (chem:make-atom :H1 :h))
(defparameter *h2* (chem:make-atom :H2 :h))
(defparameter *h3* (chem:make-atom :H3 :h))
(defparameter *h4* (chem:make-atom :H4 :h))
(defparameter *h5* (chem:make-atom :H5 :h))
(defparameter *h6* (chem:make-atom :H6 :h))
(defparameter *h7* (chem:make-atom :H7 :h))
(defparameter *h8* (chem:make-atom :H8 :h))
(defparameter *h9* (chem:make-atom :H9 :h))
(defparameter *h10* (chem:make-atom :H10 :h))
(defparameter *h11* (chem:make-atom :H11 :h))
(defparameter *h12* (chem:make-atom :H12 :h))
(defparameter *s1* (chem:make-atom :S1 :s))

(chem:add-matter *res* *c1*)
(chem:add-matter *res* *c2*)
(chem:add-matter *res* *c3*)
(chem:add-matter *res1* *c4*)
(chem:add-matter *res1* *c5*)
(chem:add-matter *res1* *c6*)
(chem:add-matter *res* *o1*)
(chem:add-matter *res* *o2*)
(chem:add-matter *res1* *o3*)
(chem:add-matter *res* *n1*)
(chem:add-matter *res1* *n2*)
(chem:add-matter *res* *h1*)
(chem:add-matter *res* *h2*)
(chem:add-matter *res* *h3*)
(chem:add-matter *res* *h4*)
(chem:add-matter *res* *h5*)
(chem:add-matter *res* *h6*)
(chem:add-matter *res1* *h7*)
(chem:add-matter *res1* *h8*)
(chem:add-matter *res1* *h9*)
(chem:add-matter *res1* *h10*)
(chem:add-matter *res1* *h11*)
(chem:add-matter *res1* *h12*)
(chem:add-matter *res* *s1*)

(chem:bond-to *h1* *o1* :single-bond)
(chem:bond-to *o1* *c1* :single-bond)
(chem:bond-to *c1* *o2* :double-bond)
(chem:bond-to *c1* *c2* :single-bond)
(chem:bond-to *c2* *h3* :single-bond)
(chem:bond-to *c2* *c3* :single-bond)
(chem:bond-to *c3* *h4* :single-bond)
(chem:bond-to *c3* *h5* :single-bond)
(chem:bond-to *c3* *s1* :single-bond)
(chem:bond-to *s1* *h2* :single-bond)
(chem:bond-to *c2* *n1* :single-bond)
(chem:bond-to *n1* *h6* :single-bond)
(chem:bond-to *n1* *c4* :single-bond)
(chem:bond-to *c4* *o3* :double-bond)
(chem:bond-to *c4* *c5* :single-bond)
(chem:bond-to *c5* *h7* :single-bond)
(chem:bond-to *c5* *c6* :single-bond)
(chem:bond-to *c6* *h8* :single-bond)
(chem:bond-to *c6* *h9* :single-bond)
(chem:bond-to *c6* *h10* :single-bond)
(chem:bond-to *c5* *n2* :single-bond)
(chem:bond-to *n2* *h11* :single-bond)
(chem:bond-to *n2* *h12* :single-bond)
(defparameter *agg* (chem:make-aggregate))
(chem:add-matter *agg* *met*)


NIL

In [17]:
(energy::setup-amber)

SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


#<FORCE-FIELD >

In [18]:
(cando:jostle *met* 80)
(defparameter *me* (energy:minimize *agg*))

energy-nonbond -> #<ENERGY-NONBOND >
ffnonbond-db -> #<FFNONBOND-DB @0x11fc440b8>
type1 0 type2 0
type1 0 type2 3
type1 0 type2 49
type1 0 type2 48
type1 0 type2 35
type1 0 type2 26
type1 0 type2 28
type1 0 type2 18
type1 0 type2 24
type1 0 type2 25
type1 0 type2 67
type1 0 type2 38
type1 3 type2 3
type1 3 type2 49
type1 3 type2 48
type1 3 type2 35
type1 3 type2 26
type1 3 type2 28
type1 3 type2 18
type1 3 type2 24
type1 3 type2 25
type1 3 type2 67
type1 3 type2 38
type1 49 type2 49
type1 49 type2 48
type1 49 type2 35
type1 49 type2 26
type1 49 type2 28
type1 49 type2 18
type1 49 type2 24
type1 49 type2 25
type1 49 type2 67
type1 49 type2 38
type1 48 type2 48
type1 48 type2 35
type1 48 type2 26
type1 48 type2 28
type1 48 type2 18
type1 48 type2 24
type1 48 type2 25
type1 48 type2 67
type1 48 type2 38
type1 35 type2 35
type1 35 type2 26
type1 35 type2 28
type1 35 type2 18
type1 35 type2 24
type1 35 type2 25
type1 35 type2 67
type1 35 type2 38
type1 26 type2 26
type1 26 type2 28
type1 26

*ME*

In [21]:
(defparameter *mol2* (chem:make-molecule))


*MOL2*

In [63]:
(defparameter *mol1* (chem:make-molecule))
(defparameter *mol2* (chem:make-molecule))
(defparameter *res1* (chem:make-residue :foo))
(defparameter *res2* (chem:make-residue :bar))

(defparameter *a1* (chem:make-atom :CL :|Cl|))
(defparameter *a2* (chem:make-atom :CL :|Cl|))

(chem:set-position *a1* '(0.0 0.0 0.0))
(chem:set-position *a2* '(3.0 0.0 0.0))
(chem:set-charge *a1* '1.0)
(chem:set-charge *a2* '1.0)
(chem:add-matter *res1* *a1*)
(chem:add-matter *res2* *a2*)
(chem:add-matter *mol1* *res1*)
(chem:add-matter *mol2* *res2*)

(defparameter *agg* (chem:make-aggregate))
(chem:add-matter *agg* *mol1*)
(chem:add-matter *agg* *mol2*)                                                                         
(chem:set-property *agg* :solvent-box '(20.0 30.0 40.0))
(energy::setup-amber)


SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


#<FORCE-FIELD >

In [66]:
(nglv::show-aggregate *agg*)


A Jupyter Widget